# Intro

This notebook demonstrates the use of the tweeteater functions. The goal of the tweeteater library is to provide functions that allow simple loading of large Twitter datasets collected from the streaming API. It particularly allows to extract nested observations of engagements.

# Parameters

In [ ]:
# the directory with the tweets 
directory = 'test_data'
extension = '.jsonl'
filepaths = get_filepaths(directory, extension)
n_filepaths = len(list(get_filepaths(directory, extension)))
ids = [id_ for id_, types in load_pickle('test_data/tweettypes.pickle').items()]
attributes = ['created_at', 'user.screen_name']
tweet_attributes = eat_tweetattributes(filepaths, ids, attributes, n_filepaths=n_filepaths)
assert dict(tweet_attributes) == load_pickle('test_data/tweets_attributes.pickle')
